# Distributed training with horovod
Horovod is a distributed training framework based on MPI. Horovod is only available with TensorFlow version 1.12 or newer. You can find more details at [Horovod README](https://github.com/uber/horovod).

To enable Horovod, we need to make small changes to our script.

## Create a training script that support Horovod distributed training
Create a copy of the script (training_script/cifar10_keras_sm.py, **not the pipe script**) and save it as training_script/cifar10_keras_dist.py.
in:
```python
def main(args):
```

### Start horovod
add horovod support using the following code:
```python
    import horovod.keras as hvd
    hvd.init()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))
```

### Configure callbacks
add the following callbacks:
```python
    callbacks.append(hvd.callbacks.BroadcastGlobalVariablesCallback(0))
    callbacks.append(hvd.callbacks.MetricAverageCallback())
    callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1))
```

change the checkpointd and tensorboard callback to run only on `hvd.rank() == o` (You want only a single process the send logs)
```python
    if hvd.rank() == 0:
        callbacks.append(ModelCheckpoint(args.output_dir + '/checkpoint-{epoch}.h5'))
        callbacks.append(TensorBoard(log_dir=args.model_dir,update_freq='epoch'))
```

### Configure the optimizer
in
```python
# Add hvd to the function. also add it in the function call
def keras_model_fn(learning_rate, weight_decay, optimizer, momentum, hvd): 
```
configure the horovod optimizer.
Change `size=1` to `size=hvd.size()`  

add 
```python
opt = hvd.DistributedOptimizer(opt)
```
before 
```python
 model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
```

## Run Distributed training
To start a distributed training job with Horovod, configure the job distribution:
```python
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': # Number of Horovod processes per host
                        }
                }
```

Run the same job using 2 ml.p3.2xlarge instances (processes_per_host:1).  
add the distributions configuration

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [ ]:
from sagemaker.tensorflow import TensorFlow
# Change base_job_name to 'cifar10-dist' for console visibility
estimator = ... 

In [ ]:
estimator.fit()

**Good job!**  
You can now use SageMaker training jobs for distributed jobs.
Before continuing to the next notebook, look at the distribution job metrics from CloudWatch and TensorBoard.  
You can use TensorBoard to compare between the different jobs that you ran.
Run TensorBoard with   
`--logdir dist:dist_model_dir,pipe:pipe_model_dir,file:normal_job_model_dir`